## Extract Poses from Amass Dataset

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import zipfile
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm



from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [2]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
from human_body_prior.body_model.body_model import BodyModel

neutral_bm_path = './body_models/smplx/SMPLX_NEUTRAL_2020.npz'
neutral_bm = BodyModel(bm_fname=neutral_bm_path, num_betas=300, num_expressions=100).to(comp_device)
faces = c2c(neutral_bm.f)

In [4]:
import pathlib

beat2_d = pathlib.Path('./beat2_data')
motion_fs = [pathlib.Path(f'{rt}/{f}')
             for rt,ds,fs in os.walk(beat2_d)
             for f in fs
             if f.endswith('.npz')]
motion_fs.sort()
print('num files:',len(motion_fs))

num files: 2048


In [5]:
def amass_to_pose(motion_f, joints_f):
    bdata = np.load(motion_f, allow_pickle=True)
    assert bdata['mocap_frame_rate'].item() == 30
    assert bdata['model'].item() == 'smplx2020'
    assert bdata['gender'].item() == 'neutral'
    bm = neutral_bm
    T = bdata['poses'].shape[0]
    B = 1024 # batch size
    results = list()
    for i_beg in range(0,T,B):
        bdata_poses = bdata['poses'][i_beg:i_beg+B]
        bdata_trans = bdata['trans'][i_beg:i_beg+B]
        bdata_expression = bdata['expressions'][i_beg:i_beg+B]
        body_parms = {
                'root_orient': torch.Tensor(bdata_poses[:,:3]).to(comp_device),
                'pose_body'  : torch.Tensor(bdata_poses[:,3:66]).to(comp_device),
                'pose_hand'  : torch.Tensor(bdata_poses[:,75:]).to(comp_device),
                'trans'      : torch.Tensor(bdata_trans).to(comp_device),
                'betas'      : torch.Tensor(np.repeat(bdata['betas'][np.newaxis], repeats=len(bdata_trans), axis=0)).to(comp_device),
                'expression' : torch.Tensor(bdata_expression).to(comp_device)
            }    
        with torch.no_grad():
            body = bm(**body_parms)
        pose_seq_np = body.Jtr.detach().cpu().numpy()
        results.append(pose_seq_np)
    results = np.concatenate(results,axis=0)
    assert results.shape[0] == T
    np.save(joints_f,results)
    return np.isnan(results).any()

In [6]:
pose_data_d = pathlib.Path('./pose_data')
bad_zip_files = list()
for motion_f in tqdm(motion_fs,desc='processing',ncols=150):
    joints_f = pose_data_d / motion_f.relative_to(beat2_d).with_suffix('.npy')
    joints_f.parent.mkdir(parents=True,exist_ok=True)
    # motion_f,joints_f = motion_f.resolve(),joints_f.resolve()
    try:
        amass_to_pose(motion_f,joints_f)
    except zipfile.BadZipFile:
        bad_zip_files.append(motion_f)
for f in bad_zip_files:
    print('bad zip file:',f)

processing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 2048/2048 [07:03<00:00,  4.83it/s]


## Segment, Mirror and Relocate Motions

In [1]:
from collections import defaultdict
import csv
import os
import pathlib

from tqdm import tqdm
import numpy as np

bm_params_f = pathlib.Path('./body_models/smplx/SMPLX_NEUTRAL_2020.npz')
index_f = pathlib.Path('./index.csv')
pose_data_d  = pathlib.Path('./pose_data')
joints_d = pathlib.Path('./joints')

Find the corresponding left/right joints from model npy file. We will mirror left/right joints to augment data.

In [2]:
bm_params = np.load(bm_params_f,allow_pickle=True)
joint2ind = bm_params['joint2num'].item()
ind2joint = {v:k
             for k,v in joint2ind.items()}
l_joints,r_joints = list(),list()
for j in joint2ind:
    if j.startswith('L_'):
        l_j = j
        r_j = j.replace('L_','R_')
        l_joints.append(joint2ind[l_j])
        r_joints.append(joint2ind[r_j])
# print('num joints to swap:',len(l_joints))
# for l,r in sorted(zip(l_joints,r_joints)):
#     print(ind2joint[l],ind2joint[r])
joints_to_drop = [joint2ind['Jaw'],
                  joint2ind['L_Eye'],
                  joint2ind['R_Eye']]

Mirror each file and split to max sequence length of 200 each because MotionGPT uses these limits.

In [3]:
data_fs = [pathlib.Path(f'{rt}/{f}')
           for rt,ds,fs in os.walk(pose_data_d)
           for f in fs
           if f.endswith('.npy')]
data_fs.sort()
pose_data_to_joints_map = ['file,id']
for i,f in enumerate(tqdm(data_fs,desc='mirror & split files',ncols=150)):
    data = np.load(f)
    data[...,0] *= -1
    data_m = data.copy()
    data_m[:,l_joints] = data[:,r_joints]
    data_m[:,r_joints] = data[:,l_joints]
    data = np.delete(data,joints_to_drop,axis=1)
    data_m = np.delete(data_m,joints_to_drop,axis=1)
    for j,beg in enumerate(range(0,data.shape[0],200)):
        id = f'{i:06}_{j:03}'
        id_m = f'M{i:06}_{j:03}'
        out_f = joints_d / f'{id}.npy'
        out_m_f = joints_d / f'{id_m}.npy'
        if out_f.is_file() and out_m_f.is_file():
            pose_data_to_joints_map.append(f'{f},{id}')
            continue
        np.save(out_f,data[beg:beg+200])
        np.save(out_m_f,data_m[beg:beg+200])
        pose_data_to_joints_map.append(f'{f},{id}')
_ = open('pose_data_to_joints_map.txt','w').write('\n'.join(pose_data_to_joints_map) + '\n')

  0%|                                                                                                                        | 0/2048 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2048/2048 [07:21<00:00,  4.64it/s]


Write fake text files for each example. Copy a single text file from AMASS dataset to all the examples.

In [4]:
!git checkout main -- HumanML3D/texts.zip
!rm -rf HumanML3D/texts
!unzip -q HumanML3D/texts.zip -d HumanML3D
!rm HumanML3D/texts.zip
!mv HumanML3D/texts HumanML3D/texts_orig

In [5]:
import shutil

texts_d = pathlib.Path('HumanML3D/texts')
src_f = pathlib.Path('HumanML3D/texts_orig/000000.txt')
texts_d.mkdir()
data_fs = list(joints_d.iterdir())
data_fs.sort()

for f in tqdm(data_fs,'creating text files',ncols=150):
    txt_f = f.with_suffix('.txt').name
    txt_f = texts_d / txt_f
    shutil.copyfile(src_f,txt_f)

creating text files: 100%|█████████████████████████████████████████████████████████████████████████████████████| 73546/73546 [03:51<00:00, 317.83it/s]


In [6]:
!rm -r HumanML3D/texts_orig